In [1]:
# %pip install ipykernel

In [2]:
import random
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv("../data/22DEC2023_AMS_processed.csv", parse_dates=['time_sch', 'time_act'])
df['delay'] = (df.time_act - df.time_sch).dt.seconds / 60 # in minutes
df.tail(5)

,time_sch,time_act,code,dest,stat,orig,pass_load,time_diff,delay
1276,2023-12-22 23:25:00,2023-12-22 23:25:00,HV 5754 Transavia,Amsterdam,BAGGAGE HANDLED,Marrakech (RAK),280,0.0,0.0
1277,2023-12-22 23:40:00,2023-12-23 00:03:00,HV 5952 Transavia,Amsterdam,BAGGAGE HANDLED,Lisbon (LIS),291,-85020.0,23.0
1278,2023-12-22 23:40:00,2023-12-23 00:00:00,HV 6902 Transavia,Amsterdam,BAGGAGE HANDLED,Dubai International (DXB),386,-85200.0,20.0
1279,2023-12-22 23:40:00,2023-12-23 02:31:00,HV 5820 Transavia,Amsterdam,BAGGAGE HANDLED,Bari (BRI),288,-76140.0,171.0
1280,2023-12-22 23:55:00,2023-12-23 00:47:00,HV 6332 Transavia,Amsterdam,BAGGAGE HANDLED,Valencia (VLC),329,-83280.0,52.0


In [4]:
# define helper functions
def count(seq):
    """Count the number of items in sequence that are interpreted as true."""
    return sum(map(bool, seq))


def first(iterable, default=None):
    """Return the first element of an iterable; or default."""
    return next(iter(iterable), default)


def is_in(elt, seq):
    """Similar to (elt in seq), but compares with 'is', not '=='."""
    return any(x is elt for x in seq)

In [5]:
class Problem:
    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2. If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value. Hill Climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

In [6]:
# recall:
# A CSP is specified by the following inputs:
#     variables   A list of variables; each is atomic (e.g. int or string).
#     domains     A dict of {var:[possible_value, ...]} entries.
#     neighbors   A dict of {var:[var,...]} that for each variable lists
#                 the other variables that participate in constraints.
#     constraints A function f(A, a, B, b) that returns true if neighbors
#                 A, B satisfy the constraint when they have values A=a, B=b

# Modify for earliest time

The min-conflict algorithm selects a flight's domain that results in the minimum number of conflicts. If one desires to select the earliest possible time within the constraints, we have to sort the domain values for each flight and select the earliest one.

However, note that min-conflicts is a heuristic method and doesn't always find the globally optimal solution—it finds a solution that minimizes conflicts. We will implement this new change in the ```min_conflics_value``` function.

# Custom

In [7]:
class CSP(Problem):
    """This class describes finite-domain Constraint Satisfaction Problems.
    A CSP is specified by the following inputs:
        variables   A list of variables; each is atomic (e.g. int or string).
        domains     A dict of {var:[possible_value, ...]} entries.
        neighbors   A dict of {var:[var,...]} that for each variable lists
                    the other variables that participate in constraints.
        constraints A function f(A, a, B, b) that returns true if neighbors
                    A, B satisfy the constraint when they have values A=a, B=b

    In the textbook and in most mathematical definitions, the
    constraints are specified as explicit pairs of allowable values,
    but the formulation here is easier to express and more compact for
    most cases (for example, the n-Queens problem can be represented
    in O(n) space using this notation, instead of O(n^4) for the
    explicit representation). In terms of describing the CSP as a
    problem, that's all there is.

    However, the class also supports data structures and methods that help you
    solve CSPs by calling a search function on the CSP. Methods and slots are
    as follows, where the argument 'a' represents an assignment, which is a
    dict of {var:val} entries:
        assign(var, val, a)     Assign a[var] = val; do other bookkeeping
        unassign(var, a)        Do del a[var], plus other bookkeeping
        nconflicts(var, val, a) Return the number of other variables that
                                conflict with var=val
        curr_domains[var]       Slot: remaining consistent values for var
                                Used by constraint propagation routines.
    The following methods are used only by graph_search and tree_search:
        actions(state)          Return a list of actions
        result(state, action)   Return a successor of state
        goal_test(state)        Return true if all constraints satisfied
    The following are just for debugging purposes:
        nassigns                Slot: tracks the number of assignments made
        display(a)              Print a human-readable representation
    """

    def __init__(self, variables, domains, neighbors, constraints):
        """Construct a CSP problem. If variables is empty, it becomes domains.keys()."""
        super().__init__(())
        variables = variables or list(domains.keys())
        self.variables = variables
        self.domains = domains
        self.neighbors = neighbors
        self.constraints = constraints
        self.curr_domains = None
        self.nassigns = 0

    def assign(self, var, val, assignment):
        """Add {var: val} to assignment; Discard the old value if any."""
        assignment[var] = val
        self.nassigns += 1

    def unassign(self, var, assignment):
        """Remove {var: val} from assignment.
        DO NOT call this if you are changing a variable to a new value;
        just call assign for that."""
        if var in assignment:
            del assignment[var]

    def nconflicts(self, var, val, assignment):
        """Return the number of conflicts var=val has with other variables."""

        # Subclasses may implement this more efficiently
        def conflict(var2):
            return var2 in assignment and not self.constraints(var, val, var2, assignment[var2], assignment)

        return sum(conflict(v) for v in self.neighbors[var])

    def display(self, assignment):
        """Show a human-readable representation of the CSP."""
        # Subclasses can print in a prettier way, or display with a GUI
        print(assignment)

    # These methods are for the tree and graph-search interface:

    def actions(self, state):
        """Return a list of applicable actions: non conflicting
        assignments to an unassigned variable."""
        if len(state) == len(self.variables):
            return []
        else:
            assignment = dict(state)
            var = first([v for v in self.variables if v not in assignment])
            return [(var, val) for val in self.domains[var]
                    if self.nconflicts(var, val, assignment) == 0]

    def result(self, state, action):
        """Perform an action and return the new state."""
        (var, val) = action
        return state + ((var, val),)

    def goal_test(self, state):
        """The goal is to assign all variables, with all constraints satisfied."""
        assignment = dict(state)
        return (len(assignment) == len(self.variables)
                and all(self.nconflicts(variables, assignment[variables], assignment) == 0
                        for variables in self.variables))

    # These are for constraint propagation

    def support_pruning(self):
        """Make sure we can prune values from domains. (We want to pay
        for this only if we use it.)"""
        if self.curr_domains is None:
            self.curr_domains = {v: list(self.domains[v]) for v in self.variables}

    def suppose(self, var, value):
        """Start accumulating inferences from assuming var=value."""
        self.support_pruning()
        removals = [(var, a) for a in self.curr_domains[var] if a != value]
        self.curr_domains[var] = [value]
        return removals

    def prune(self, var, value, removals):
        """Rule out var=value."""
        self.curr_domains[var].remove(value)
        if removals is not None:
            removals.append((var, value))

    def choices(self, var):
        """Return all values for var that aren't currently ruled out."""
        return (self.curr_domains or self.domains)[var]

    def infer_assignment(self):
        """Return the partial assignment implied by the current inferences."""
        self.support_pruning()
        return {v: self.curr_domains[v][0]
                for v in self.variables if 1 == len(self.curr_domains[v])}

    def restore(self, removals):
        """Undo a supposition and all inferences from it."""
        for B, b in removals:
            self.curr_domains[B].append(b)

    # This is for min_conflicts search

    def conflicted_vars(self, current):
        """Return a list of variables in current assignment that are in conflict"""
        return [var for var in self.variables
                if var in current and self.nconflicts(var, current[var], current) > 0]



### The `FlightSchedulerCSP` Class

This class extends a generic Constraint Satisfaction Problem (CSP) to specifically handle flight scheduling under certain constraints like runway availability and disruption times.

#### `__init__` Method
- Initializes the flight scheduler model with the necessary parameters and calls the `CSP` class constructor.
- Parameters:
  - `flights_df`: DataFrame containing flight details.
  - `disruption_level`: Minutes of delay caused by a disruption.
  - `num_runways`: Number of available runways.
  - `neighbor_window`: The time window in minutes to consider flights as neighbors.
  - `time_slot`: The interval in minutes for generating time slots.
- It also initializes a special 'diverted' slot for handling unscheduled flights.

#### `create_variables` Method
- Creates a list of variables for the CSP. Each flight code is considered a variable.

#### `create_domains` Method
- Generates the domains (possible values) for each variable (flight). Domains are the potential time slots for each flight, considering the disruption level.
- The method also appends the `diverted_slot` to each flight's domain as a fallback option.

#### `create_neighbors` Method
- Defines neighbors for each flight based on the `neighbor_window`. Neighbors are flights scheduled close in time to each other.

#### `create_constraints` Method
- Sets up the constraints for the CSP. The constraints determine whether a particular assignment is valid.
- Key constraints include checking if assigning a time slot to a flight would exceed runway capacity and managing the diverted slot.

### Min-Conflicts Search Functions

#### `min_conflicts` Function
- A hill-climbing algorithm to solve the CSP. It iteratively tries to reduce the number of conflicts in the current assignment.
- Includes a safeguard to handle cases where a solution isn't found within the maximum steps.
- Returns the final assignment or diverts unresolved flights.

#### `min_conflicts_value` Function
- Chooses the best value (time slot) for a variable (flight) that results in the least number of conflicts.
- If there's a tie, the earliest time slot is selected. If no valid time slot is found, it resorts to the diverted slot.

### Additional Helper Functions

- `argmin_tie`: A utility function to choose the minimum element from a sequence, breaking ties based on a provided key function.

### Summary
- This custom CSP model is designed to handle a complex flight scheduling problem.
- It manages constraints like runway capacity and disruption times, and it includes a mechanism to handle flights that can't be scheduled within the given constraints by diverting them.
- The min-conflicts search algorithm is used to find a near-optimal solution, considering the constraints and the nature of the problem.
- The model is flexible enough to handle various numbers of flights, runways, and disruption scenarios.

In [12]:
class FlightSchedulerCSP(CSP):
    def __init__(self, flights_df, disruption_level=30, num_runways=1, neighbor_window=5, time_slot=5):
        """Initialize the flight scheduler CSP model."""
        self.diverted_slot = pd.Timestamp('2100-01-01')
        self.flights_df = flights_df
        self.disruption_level = disruption_level
        self.num_runways = num_runways
        self.neighbor_window = neighbor_window
        self.time_slot = time_slot

        variables = self.create_variables()
        domains = self.create_domains()
        neighbors = self.create_neighbors()
        constraints = self.create_constraints()

        CSP.__init__(self, variables, domains, neighbors, constraints)

    def create_variables(self):
        """Create variables for the CSP (each flight is a variable)."""
        return self.flights_df['code'].tolist()

    def create_domains(self):
        """Create domains for each variable based on flight schedule and disruption level."""

        # get the latest time in the dataset
        latest_time = df['time_sch'].max()

        # calculate the last possible time slot considering the disruption
        last_time_slot = latest_time + pd.Timedelta(minutes=self.disruption_level) + pd.Timedelta(minutes=self.time_slot)

        # generate a list of all X-minute time slots, extending beyond the latest time in the dataset if necessary
        extended_time_slots = pd.date_range(
            start=df['time_sch'].min(), end=last_time_slot, freq=f'{self.time_slot}T').tolist()

        domains = {}
        for index, row in df.iterrows():
            flight_start_time = row['time_sch'] + pd.Timedelta(minutes=self.disruption_level)
            flight_time_slots = [time for time in extended_time_slots if time >= flight_start_time]
            domains[row['code']] = flight_time_slots

        for flight in domains:
            domains[flight].append(self.diverted_slot)

        return domains

    def create_neighbors(self):
        """Create neighbors for each variable. Returns all flights"""
        def find_neighbors(flight_code, flights_df, time_window=pd.Timedelta(minutes=self.neighbor_window)):
            flight_time = flights_df[flights_df['code'] == flight_code]['time_sch'].iloc[0]
            time_window_start = flight_time - time_window
            time_window_end = flight_time + time_window

            # find flights within the time window (excluding the flight itself)
            neighbor_flights = flights_df[
                (flights_df['time_sch'] >= time_window_start) & 
                (flights_df['time_sch'] <= time_window_end) &
                (flights_df['code'] != flight_code)
            ]

            return neighbor_flights['code'].tolist()

        variables = self.create_variables()  # Make sure this is called before using 'variables'
        return {flight: find_neighbors(flight, self.flights_df) for flight in variables}

    def create_constraints(self):
        """Define the constraint function."""
        def constraints(A, a, B, b, assignment):

            if a == self.diverted_slot or b == self.diverted_slot:
                return True  # always allow diverted slot

            # assuming runway capacity as before
            runway_capacity = self.num_runways # 3 ** (self.num_runways - 1)
            
            # Time buffer in minutes
            # time_buffer = pd.Timedelta(minutes=self.time_window)

            # check if A and B are scheduled too close to each other
            # if abs(a - b) < time_buffer and A != B:
            if a == b:
                # count flights at both time slots
                flights_at_a = sum(1 for flight, time in assignment.items() if time == a)
                flights_at_b = sum(1 for flight, time in assignment.items() if time == b)
                
                # check against runway capacity
                if flights_at_a >= runway_capacity or flights_at_b >= runway_capacity:
                    return False

            return True

        return constraints


# Min-conflicts Hill Climbing search for CSPs
def min_conflicts(csp, max_steps=100_000, resolve_strategy="early"):
    """Solve a CSP by stochastic Hill Climbing on the number of conflicts.
    Parameters:
    - csp: The CSP instance.
    - max_steps: Maximum number of steps for the algorithm.
    - resolve_strategy: Strategy to resolve conflicts ('early', 'force', 'divert').
    """
    # Generate a complete assignment for all variables (probably with conflicts)
    csp.current = current = {}
    for var in csp.variables:
        val = min_conflicts_value(csp, var, current)
        csp.assign(var, val, current)

    # Now repeatedly choose a random conflicted variable and change it
    for i in range(max_steps):
        # if i % 100 == 0: print(f"Step: {i:,}")
        conflicted = csp.conflicted_vars(current)
        if not conflicted:
            return current

        # Safeguard against getting stuck at local minima
        if resolve_strategy == "early":
            if len(current) == len(csp.variables): 
                return current
        elif resolve_strategy == "force" and i > 500:
            for flight in conflicted:
                # choose the minimum available time slot
                current[flight] = min(csp.domains[flight])
                csp.assign(flight, current[flight], current)
            return current
        elif resolve_strategy == "divert" and i > 500:
            for flight in conflicted:
                # assign to the diverted slot
                csp.assign(flight, csp.diverted_slot, current)
            return current

        var = random.choice(conflicted)
        val = min_conflicts_value(csp, var, current)
        csp.assign(var, val, current)

    return None

def min_conflicts_value(csp, var, current):
    """Return the value that will give var the least number of conflicts.
    If there is a tie, choose the earliest time."""
    # sort the domain values by time, then by the number of conflicts
    domain_sorted_by_time_and_conflicts = sorted(
        csp.domains[var],
        key=lambda val: (csp.nconflicts(var, val, current), val)
    )
    # now select the value that minimizes conflicts and is the earliest in time
    if domain_sorted_by_time_and_conflicts:
        return domain_sorted_by_time_and_conflicts[0]  # the first element captures this info
    return csp.diverted_slot

# def min_conflicts_value(csp, var, current):
#     """Return the value that will give var the least number of conflicts.
#     If there is a tie, choose the earliest time."""
#     # print(f"var: {var} \ndomains: {csp.domains[var]} \nconflicts: {csp.nconflicts(var, csp.domains[var], current)}")
#     return argmin_tie(csp.domains[var], key=lambda val: csp.nconflicts(var, val, current))


identity = lambda x: x


def argmin_tie(seq, key=identity):
    """Return a minimum element of seq; break ties at random."""
    return min(seq, key=key)


In [16]:
sample = df.tail(100) # using tail uses the first element as the basis of disruption i.e. resumption time
flight_schedule_csp = FlightSchedulerCSP(sample, disruption_level=30, num_runways=2, neighbor_window=5, time_slot=5)
solutions = min_conflicts(flight_schedule_csp, resolve_strategy="early")

sample['new_sch'] = solutions.values()
sample['new_delay'] = (sample.new_sch - sample.time_sch).dt.seconds / 60 # in minutes

print(f"Total new delay: {sample.new_delay.sum()}")
print(f"Total diverted flights: {len(sample[sample.new_sch == flight_schedule_csp.diverted_slot])}")
sample[["code", "time_sch", "new_sch", "delay", "new_delay"]]

Step: 0
Total new delay: 4510.0
Total diverted flights: 0


,code,time_sch,new_sch,delay,new_delay
1181,KL 1999 KLM,2023-12-22 21:00:00,2023-12-22 21:30:00,171.0,30.0
1182,EZS 1518 easyJet Switzerland,2023-12-22 21:05:00,2023-12-22 21:35:00,62.0,30.0
1183,SK 553 Scandinavian Airlines,2023-12-22 21:05:00,2023-12-22 21:35:00,0.0,30.0
1184,KL 1024 KLM,2023-12-22 21:05:00,2023-12-22 21:40:00,66.0,35.0
1185,KL 1915 KLM,2023-12-22 21:05:00,2023-12-22 21:40:00,45.0,35.0
...,...,...,...,...,...
1276,HV 5754 Transavia,2023-12-22 23:25:00,2023-12-23 00:00:00,0.0,35.0
1277,HV 5952 Transavia,2023-12-22 23:40:00,2023-12-23 00:10:00,23.0,30.0
1278,HV 6902 Transavia,2023-12-22 23:40:00,2023-12-23 00:10:00,20.0,30.0
1279,HV 5820 Transavia,2023-12-22 23:40:00,2023-12-23 00:15:00,171.0,35.0


# Analysis

The modifications made to the FlightSchedulerCSP class, including the addition of a diverted_slot and adjustments to time slot generation and constraints, provides in a custom flight scheduling CSP model. Let's summarize the key aspects of this experiment and compare it with tree-based search algorithms using utility functions.
Summary of the Experiment:

- Custom Time Slot Frequency: The time_slot parameter allows you to set the frequency of time slots. This change enables more precise scheduling.

- Extended Time Slot Range: The time slots are generated up to the last possible time considering the disruption level and the time slot interval. This ensures a comprehensive range for rescheduling flights.

- Diverted Slot for Unsolvable Conflicts: A diverted_slot set to a future date acts as a fallback for flights that cannot be accommodated. This helps in managing unsolvable conflicts.

- Constraints Adjustments: The constraint function now accounts for the number of flights scheduled at a time and compares it with the available runway capacity. It also accommodates the diverted slot.

- Handling of Local Minima: The min-conflicts algorithm includes a check to prevent getting stuck in local minima, ensuring that the algorithm provides a solution.

- Selection of Earliest Time Slot: The algorithm selects the earliest time slot that minimizes conflicts, or if not feasible, assigns the flight to the diverted slot.

Pros of the CSP Implementation:

- Flexibility in Scheduling: Customizable time slot intervals offer more control over the scheduling process.
- Efficient Handling of Conflicts: The ability to divert flights provides a clear solution for otherwise unsolvable scheduling conflicts.
- Adaptability to Various Scenarios: The CSP model can adapt to different runway capacities and disruption levels.
- Prevention of Stagnation: Safeguards against local minima ensure the algorithm always returns a result.

Cons of the CSP Implementation:

- Increased Complexity: The added features and parameters make the model more complex and potentially harder to manage.
- Risk of Suboptimal Solutions: The use of a diverted slot, while practical, might lead to suboptimal scheduling outcomes.
- Dependence on Parameter Settings: The effectiveness of the model heavily relies on the chosen values for parameters like time_slot and neighbor_window.

Comparison with Tree-Based Search Using Utility Functions:

- CSP vs. Tree-Based Search: CSP focuses on finding a solution that satisfies all constraints, while tree-based search algorithms aim to find an optimal solution based on a utility function.
- Handling of Conflicts: CSPs, especially with the addition of the diverted slot, can handle conflicts more flexibly compared to tree-based searches, which might require more complex utility functions to manage similar scenarios.
- Scalability and Performance: Tree-based searches can be more computationally intensive, especially for large problem spaces, whereas CSPs with the min-conflicts heuristic can be more efficient in finding satisfactory solutions.
- Optimality of Solutions: Tree-based searches are more likely to find the most optimal solution, but at the cost of increased computational effort. In contrast, CSPs may quickly find a satisfactory solution but not necessarily the most optimal one.

In summary, the CSP model is capable to handle large state spaces quickly. While it offers a practical approach to managing unsolvable conflicts, it may not always provide the most optimal scheduling compared to tree-based search algorithms focused on maximizing a utility function.

# Conflict Resolution
Early Termination Strategy ("early"):

Description: This strategy stops the algorithm once all flights have an assigned time slot, even if conflicts still exist. It doesn't necessarily mean a globally conflict-free solution, but rather that each flight has at least one time slot assigned.

Pros:
- Quick Results: It provides a solution faster, as it doesn't insist on resolving all conflicts.
- Practicality: It can be practical in real-world scenarios where perfect solutions are rare, and some level of conflict is acceptable.
Cons:
- Possible Conflicts: There may still be unresolved conflicts, requiring manual intervention or further decision-making.
- Suboptimal Assignments: Some flights might be assigned to less desirable time slots just to ensure every flight has a slot.

Forced Resolution Strategy ("force"):

Description: This strategy forces a resolution after a certain number of steps (500 in your case) by assigning each conflicted flight to its earliest possible time slot.

Pros:
- Conflict Resolution: It actively reduces conflicts by forcing assignment of conflicted flights to their earliest slots.
- Balance: Offers a balance between efficiency and thorough conflict resolution.
Cons:
- Suboptimal Timing: Might assign flights to times that are too early or inconvenient, disregarding preferences.
- Arbitrary Cutoff: The step count (500) at which this strategy kicks in is arbitrary and may not suit all datasets or scenarios.

Diversion Strategy ("divert"):

Description: Similar to the forced resolution strategy, but here, after a set number of steps, all remaining conflicted flights are assigned to a 'diverted' slot, effectively removing them from the normal schedule.

Pros:
- Clear Conflict Resolution: All conflicts are resolved by removing problematic flights from the regular schedule.
- Simplicity: Simplifies the decision-making process as it provides a clear action for unresolved conflicts.
Cons:
- Loss of Options: Diverting flights can be seen as a last resort and might not always be the most practical or desirable solution.
- Over-Simplification: May not reflect the complexity and nuances of real-world flight scheduling, where complete diversion might be impractical.

Overall Analysis:

- Adaptability: Each strategy has its merits and is adaptable to different operational constraints and objectives. The choice of strategy should be influenced by the specific requirements of the scheduling context (e.g., airport capacity, passenger convenience, operational costs).
- Implementation Complexity: The implementation of these strategies is relatively straightforward, but the choice of strategy can significantly impact the practical utility of the solutions generated.
- Optimality vs. Practicality: There's a trade-off between seeking an optimal solution (less conflict) and a practical solution (quick assignment with manageable conflict). The chosen strategy should align with the operational goals of the flight scheduler.
- Manual Intervention: In all cases, especially with "early" and "divert" strategies, some manual review or additional decision-making might be necessary to handle the assigned schedules effectively.

In real-world applications, these strategies might be used in combination or in different phases of problem-solving, depending on the operational priorities and constraints at hand.

Conclusion:

- Choose Early if your primary goal is to quickly assign time slots to all flights, acknowledging that some level of conflict or suboptimal scheduling might remain.
- Choose Divert if operational simplicity, clear conflict resolution, and realism are your primary concerns, and you're equipped to handle the logistical and customer service challenges of diverted flights.
- Choose Force if minimizing disruptions and maintaining passenger satisfaction are more critical, and you're prepared to manage the complexities and potential resource strains that come with tightly packed schedules.

Ultimately, the best option depends on your specific operational priorities, resource capabilities, and the level of flexibility you have in managing scheduling challenges.